In [10]:
# !pip install ipdb, nltk, visdom, torchnet

In [1]:
import ipdb
import sys

In [2]:
sys.path.append('C:/Users/SEONGGYUN/ATAE-LSTM/ATAE-LSTM')
import Ipynb_importer
from config import opt
from data.Embedding import Emb
from data.AspClas import AspClas
from models.ATAE_LSTM import ATAE_LSTM
from utils.visualize import Visualizer
from config import opt
from tqdm import tqdm
from random import randint
import numpy as np

importing Jupyter notebook from c:\Users\SEONGGYUN\ATAE-LSTM\ATAE-LSTM\data\Embedding.ipynb


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SEONGGYUN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


importing Jupyter notebook from c:\Users\SEONGGYUN\ATAE-LSTM\ATAE-LSTM\data\AspClas.ipynb
importing Jupyter notebook from c:\Users\SEONGGYUN\ATAE-LSTM\ATAE-LSTM\models\ATAE_LSTM.ipynb
importing Jupyter notebook from c:\Users\SEONGGYUN\ATAE-LSTM\ATAE-LSTM\models\BasicModule.ipynb


In [3]:
import torch as t
from torch.utils.data import DataLoader
from torchnet import meter
from torch.autograd import Variable

In [4]:
def val(model, dataloader):
    '''
    计算模型在验证集上的准确率等信息
    '''
    
    confusion_matrix = meter.ConfusionMeter(opt.classes)
    model.eval()
    with t.no_grad():
        for step, data in enumerate(dataloader):
            sentence, terms, label = data
            #if opt.use_cuda:
                #sentence, terms, label = sentence.cuda(), terms.cuda(), label.cuda()
            score = model(sentence, terms)
            confusion_matrix.add(score.data.cpu(), label.data.cpu().squeeze())
    model.train()
    cm_value = confusion_matrix.value()
    accuracy = 100.0 * (cm_value.trace()) / (cm_value.sum())
    class_equal_accuracy = (float(cm_value[0][0])/(cm_value[0].sum())
        +float(cm_value[1][1])/(cm_value[1].sum())
        +float(cm_value[2][2])/(cm_value[2].sum())
    ) * 100 / 3
    return confusion_matrix, accuracy, class_equal_accuracy

In [5]:
!python -m visdom.server
vis = Visualizer(opt.env)

Traceback (most recent call last):
  File "c:\Users\SEONGGYUN\anaconda3\envs\modeling23-2\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\Users\SEONGGYUN\anaconda3\envs\modeling23-2\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\Users\SEONGGYUN\anaconda3\envs\modeling23-2\lib\site-packages\visdom\server\__main__.py", line 16, in <module>
    download_scripts_and_run()
  File "c:\Users\SEONGGYUN\anaconda3\envs\modeling23-2\lib\site-packages\visdom\server\run_server.py", line 236, in download_scripts_and_run
    main()
  File "c:\Users\SEONGGYUN\anaconda3\envs\modeling23-2\lib\site-packages\visdom\server\run_server.py", line 230, in main
    eager_data_loading=FLAGS.eager_data_loading,
  File "c:\Users\SEONGGYUN\anaconda3\envs\modeling23-2\lib\site-packages\visdom\server\run_server.py", line 55, in start_server
    app.listen(port, max_buffer_size=1024**3)
  File "c:\Users\SEONGGYUN\anaconda3\envs\modeling23-2\lib\site-pac

Checking for scripts.
It's Alive!


In [6]:
# step1 data
train_data = AspClas(opt.train_data_root, train=True)
test_data = AspClas(opt.train_data_root, train=False, emb=train_data.emb)
train_dataloader = DataLoader(
    train_data,
    opt.batch_size,
    shuffle=True,
    drop_last = True
)
test_dataloader = DataLoader(
    test_data,
    opt.batch_size,
    shuffle=False,
    drop_last = True
)
words = train_data.emb._get_words_()

100%|██████████| 100000/100000 [00:06<00:00, 15177.16it/s]


Embedding : successfully input 100000 pretrained word embeddings while 0 failed


100%|██████████| 3044/3044 [00:00<00:00, 594803.70it/s]


In [7]:
# step2 configure model
model = ATAE_LSTM(emb=train_data.emb)
#if opt.use_cuda:
    #model = model.cuda()

In [8]:
# step3 criterion and optimizer
if opt.rescaling:
    class_weights = t.Tensor([3,4,1]) # weights on every class
    #if opt.use_cuda:
        #class_weights = class_weights.cuda()
else:
    class_weights = None
criterion = t.nn.CrossEntropyLoss(weight = class_weights)
lr = opt.lr
optimizer = t.optim.Adam(
    model.parameters(),
    lr = lr,
    weight_decay = opt.weight_decay
)

In [9]:
# step4 meters
loss_meter = meter.AverageValueMeter()
confusion_matrix = meter.ConfusionMeter(opt.classes)
previous_loss = 1e100
best_val_accuracy = 0

In [10]:
def print_attention(model, words, test_dataloader=None, sentence_terms_label=None):
    if test_dataloader is not None:
        sentence, terms, label = list(test_dataloader)[0]
        #if opt.use_cuda:
            #sentence, terms, label = sentence.cuda(), terms.cuda(), label.cuda()
    else:
        (sentence, terms, label) = sentence_terms_label
    tokens = [words[i] for i in sentence[0].tolist() if i!=0]
    term = [words[i] for i in terms[0].tolist() if i!=0]
    score, attention = model(sentence, terms, returnAttention=True)
    attention_probs = attention[0][0][:len(tokens)].tolist()
    tokens_attention = [(tokens[i], "%.3f"%attention_probs[i]) for i in range(len(tokens))]
    tqdm.write(str(tokens))
    tqdm.write(str(term))
    tqdm.write(str(tokens_attention))

In [12]:
# step5 train

# validate and visualize at start
val_cm, val_accuracy, class_equal_accuracy = val(model, test_dataloader)
vis.plot('val_accuracy', val_accuracy)
vis.plot('val_class_equal_accuracy', class_equal_accuracy)
vis.plot('lr', lr)
vis.log("epoch:{epoch},\nlr:{lr},\ntrain_cm:{train_cm},\nval_cm:{val_cm}".format(
    epoch = 0,
    val_cm = str(val_cm.value()),
    train_cm=str(confusion_matrix.value()),
    lr=lr
))

total_step = 0
for epoch in tqdm(range(opt.max_epoch)):
    loss_meter.reset()
    confusion_matrix.reset()
    
    for step, (sentence, terms, label) in enumerate(train_dataloader):
        
        #if opt.use_cuda:
            #sentence, terms, label = sentence.cuda(), terms.cuda(), label.cuda()
        
        score = model(sentence, terms)
        loss = criterion(score, label.squeeze())
        loss.backward()
        optimizer.step()
        
        # meters update and visualize
        if opt.use_cuda:
            loss_meter.add(loss.data.cpu())
            confusion_matrix.add(score.data.cpu(), label.data.cpu().squeeze())
        else:
            loss_meter.add(loss.data)
            confusion_matrix.add(score.data, label.data.squeeze())
        if total_step%opt.print_freq == 0:
            vis.plot('loss', loss_meter.value()[0])
            """vis.log("score:{score},target:{label}".format(
                score = score,
                label = label
            ))"""
        total_step += 1
        
    
    # validate and visualize
    val_cm, val_accuracy, class_equal_accuracy = val(model, test_dataloader)
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        model.save(acc="%.2f"%val_accuracy)
    
    vis.plot('train_accuracy', 100.0*confusion_matrix.value().trace()/confusion_matrix.value().sum())
    vis.plot('val_accuracy', val_accuracy)
    vis.plot('val_class_equal_accuracy', class_equal_accuracy)
    vis.plot('lr', lr)
    vis.uplog("epoch:{},\nlr:{},\nval_cm:\n{}\n".format(
        epoch,lr,str(val_cm.value())
    ).replace("\n", "<br>"))
    
    # update learning rate
    if loss_meter.value()[0].item() >= previous_loss and lr>opt.lr_min:
        lr = lr * opt.lr_decay
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    previous_loss = loss_meter.value()[0]
    if epoch-1 % 100 == 0:
        opt.lr_min *= 2/3
    """
    lr = opt.lr * (np.cos(np.pi*(epoch/opt.max_epoch))+1)/2
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    """
    if epoch%1 == 0:   ### 원래 50으로 나눔 ####
        print_attention(model, words, test_dataloader=test_dataloader)
        print_attention(model, words, sentence_terms_label = (sentence, terms, label))

  0%|          | 0/10 [02:15<?, ?it/s]

['the', 'ambience', 'is', 'very', 'romantic', 'and', 'definitely', 'a', 'good', 'place', 'to', 'bring', 'a', 'date', '.']
['place']
[('the', '0.162'), ('ambience', '0.175'), ('is', '0.102'), ('very', '0.068'), ('romantic', '0.040'), ('and', '0.051'), ('definitely', '0.043'), ('a', '0.037'), ('good', '0.045'), ('place', '0.058'), ('to', '0.061'), ('bring', '0.056'), ('a', '0.039'), ('date', '0.043'), ('.', '0.019')]


 10%|█         | 1/10 [02:16<20:29, 136.65s/it]

['anyway', ',', 'the', 'food', 'is', 'good', ',', 'the', 'price', 'is', 'right', 'and', 'they', 'have', 'a', 'decent', 'wine', 'list', '.']
['price']
[('anyway', '0.084'), (',', '0.091'), ('the', '0.092'), ('food', '0.170'), ('is', '0.056'), ('good', '0.039'), (',', '0.045'), ('the', '0.053'), ('price', '0.047'), ('is', '0.032'), ('right', '0.021'), ('and', '0.023'), ('they', '0.025'), ('have', '0.023'), ('a', '0.024'), ('decent', '0.041'), ('wine', '0.059'), ('list', '0.049'), ('.', '0.024')]


 10%|█         | 1/10 [04:23<20:29, 136.65s/it]

...

['the', 'food', 'inludes', 'famous', 'scrumptious', 'bombay', 'style', 'chaat', 'such', 'as', 'bhelpuri', ',', 'sevpuri', 'and', 'samosa', 'chaats', ',', 'as', 'well', 'as', 'other', 'great', 'indian', 'appetizers', '.']
['food']
[('the', '0.056'), ('food', '0.075'), ('inludes', '0.062'), ('famous', '0.068'), ('scrumptious', '0.051'), ('bombay', '0.033'), ('style', '0.038'), ('chaat', '0.038'), ('such', '0.039'), ('as', '0.035'), ('bhelpuri', '0.041'), (',', '0.025'), ('sevpuri', '0.033'), ('and', '0.029'), ('samosa', '0.037'), ('chaats', '0.038'), (',', '0.025'), ('as', '0.030'), ('well', '0.032'), ('as', '0.032'), ('other', '0.040'), ('great', '0.030'), ('indian', '0.043'), ('appetizers', '0.060'), ('.', '0.009')]


 90%|█████████ | 9/10 [22:03<02:15, 135.22s/it]

['the', 'ambience', 'is', 'very', 'romantic', 'and', 'definitely', 'a', 'good', 'place', 'to', 'bring', 'a', 'date', '.']
['place']
[('the', '0.109'), ('ambience', '0.124'), ('is', '0.087'), ('very', '0.067'), ('romantic', '0.064'), ('and', '0.054'), ('definitely', '0.069'), ('a', '0.051'), ('good', '0.049'), ('place', '0.063'), ('to', '0.063'), ('bring', '0.074'), ('a', '0.052'), ('date', '0.052'), ('.', '0.020')]


100%|██████████| 10/10 [22:04<00:00, 132.45s/it]

['the', 'waiter', 'actually', 'poured', 'water', 'on', 'my', 'hand', 'and', 'walked', 'away', '.']
['waiter']
[('the', '0.070'), ('waiter', '0.062'), ('actually', '0.085'), ('poured', '0.072'), ('water', '0.090'), ('on', '0.101'), ('my', '0.104'), ('hand', '0.100'), ('and', '0.102'), ('walked', '0.072'), ('away', '0.088'), ('.', '0.054')]
